In [1]:
import pandas as pd
import numpy as np
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# plt.style.use('seaborn')
# sns.set(font_scale=2.5)
import missingno as msno
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# train_origin = pd.read_csv("../../main/data/origin/train.csv")
# df_cctv_origin = pd.read_csv("./data/external_open/대구 CCTV 정보.csv", encoding='cp949')
# df_light_origin = pd.read_csv("./data/external_open/대구 보안등 정보.csv", encoding='cp949')
# df_child_origin = pd.read_csv("./data/external_open/대구 어린이 보호 구역 정보.csv", encoding='cp949')
# df_parking_origin = pd.read_csv("./data/external_open/대구 주차장 정보.csv", encoding='cp949')
# df_rain_info = pd.read_csv("./data/rain_info.csv")

# train 전처리

In [3]:
train_origin = pd.read_csv("../../main/data/origin/train.csv")

In [4]:
df_cctv_origin = pd.read_csv("./data/external_open/대구 CCTV 정보.csv", encoding='cp949')

In [5]:
df_train = copy.deepcopy(train_origin)
df_train = df_train[['사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형','ECLO']]
if '시군구' in df_train.columns:
  # 군, 구(동)
  df_train[['군', '구(동)']] = df_train['시군구'].str.split(' ', expand=True)[[1,2]]
  df_train.drop(columns=['시군구'], inplace=True)

# 사고일시 yyyy-mm-dd hh datetime - 시간 컬럼 생성
if '사고일시' in df_train.columns:
  df_train['년도'] = df_train['사고일시'].str.split('-').str[0].astype(int)
  df_train['월'] = df_train['사고일시'].str.split('-').str[1].astype(int)
  df_train['시간'] = df_train['사고일시'].str.split(' ').str[-1].astype(int)
  df_train.drop(columns=['사고일시'], inplace=True)

In [6]:
df_train

,요일,기상상태,도로형태,노면상태,사고유형,ECLO,군,구(동),년도,월,시간
0,화요일,맑음,단일로 - 기타,건조,차대사람,5,중구,대신동,2019,1,0
1,화요일,흐림,단일로 - 기타,건조,차대사람,3,달서구,감삼동,2019,1,0
2,화요일,맑음,단일로 - 기타,건조,차대사람,3,수성구,두산동,2019,1,1
3,화요일,맑음,단일로 - 기타,건조,차대차,5,북구,복현동,2019,1,2
4,화요일,맑음,단일로 - 기타,건조,차대차,3,동구,신암동,2019,1,4
...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,교차로 - 교차로안,건조,차대차,3,수성구,수성동3가,2021,12,19
39605,금요일,맑음,단일로 - 기타,건조,차대차,3,달서구,상인동,2021,12,19
39606,금요일,맑음,교차로 - 교차로안,건조,차대차,10,달서구,월성동,2021,12,21
39607,금요일,맑음,기타 - 기타,건조,차대차,3,달서구,장동,2021,12,22


# CCTV

In [7]:
df_cctv = copy.deepcopy(df_cctv_origin)
df_cctv.head()

,무인교통단속카메라관리번호,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,설치장소,단속구분,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치연도
0,주차12,대구광역시,남구,시도,NaN,대명로,3,대구광역시 남구 대명로 162,대구광역시 남구 대명동 903-13,35.838857,128.575418,안지랑네거리1 버스정류장,4,0,NaN,NaN,99.0,2010.0
1,주차13,대구광역시,남구,시도,NaN,봉덕로,3,대구광역시 남구 봉덕로 135,대구광역시 남구 봉덕동 853-8,35.844653,128.604619,효성백년가약,4,0,NaN,NaN,99.0,2013.0
2,주차14,대구광역시,남구,시도,NaN,대봉로,3,대구광역시 남구 봉덕로 134,대구광역시 남구 봉덕동 866-46,35.844183,128.604815,강변코오롱하늘채,4,0,NaN,NaN,99.0,2013.0
3,주차15,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 200,대구광역시 남구 대명동 2680-17,35.834798,128.585284,보성청록타운,4,0,NaN,NaN,99.0,2014.0
4,주차16,대구광역시,남구,시도,NaN,큰골길,3,대구광역시 남구 큰골길 243,대구광역시 남구 봉덕동 1616,35.832728,128.588324,환경차고지,4,0,NaN,NaN,99.0,2014.0


In [8]:
df_cctv.dtypes

무인교통단속카메라관리번호     object
시도명               object
시군구명              object
도로종류              object
도로노선번호            object
도로노선명             object
도로노선방향             int64
소재지도로명주소          object
소재지지번주소           object
위도               float64
경도               float64
설치장소              object
단속구분               int64
제한속도               int64
단속구간위치구분         float64
과속단속구간길이         float64
보호구역구분           float64
설치연도             float64
dtype: object

In [9]:
df_cctv['도로노선번호'].value_counts()

도로노선번호
5      59
30     44
4      19
1      13
25     11
       ..
23      1
35      1
19      1
66      1
923     1
Name: count, Length: 67, dtype: int64

In [10]:
for col in df_cctv.columns:
    msg = 'column: {:>20}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_cctv[col].isnull().sum() / df_cctv[col].shape[0]))
    print(msg)

column:        무인교통단속카메라관리번호	 Percent of NaN value: 0.00%
column:                  시도명	 Percent of NaN value: 0.00%
column:                 시군구명	 Percent of NaN value: 0.00%
column:                 도로종류	 Percent of NaN value: 0.00%
column:               도로노선번호	 Percent of NaN value: 72.96%
column:                도로노선명	 Percent of NaN value: 0.00%
column:               도로노선방향	 Percent of NaN value: 0.00%
column:             소재지도로명주소	 Percent of NaN value: 15.02%
column:              소재지지번주소	 Percent of NaN value: 0.19%
column:                   위도	 Percent of NaN value: 0.00%
column:                   경도	 Percent of NaN value: 0.00%
column:                 설치장소	 Percent of NaN value: 0.00%
column:                 단속구분	 Percent of NaN value: 0.00%
column:                 제한속도	 Percent of NaN value: 0.00%
column:             단속구간위치구분	 Percent of NaN value: 95.02%
column:             과속단속구간길이	 Percent of NaN value: 97.56%
column:               보호구역구분	 Percent of NaN value: 16.15%
column:  

In [11]:
df_cctv['소재지지번주소'].isnull().sum()

2

In [12]:
df_cctv['소재지지번주소'].value_counts()

소재지지번주소
대구광역시 수성구 파동 546             5
대구광역시 달성군 화원읍 본리리 331-2      4
대구광역시 달성군 가창면 냉천리 1-2        4
대구광역시 달성군 논공읍 본리리 산 183-9    4
대구광역시 달성군 다사읍 세천리 1578-9     4
                            ..
대구광역시 달서구 월암동 1-212          1
대구광역시 수성구 두산동 595-6          1
대구광역시 수성구 만촌동 794-3          1
대구광역시 수성구 만촌동 686-2          1
대구광역시 달서구 두류동 147-7          1
Name: count, Length: 918, dtype: int64

In [13]:
if '소재지지번주소' in df_cctv.columns:
  # 군, 구(동)
  df_cctv['구(동)'] = df_cctv['소재지지번주소'].str.split(' ', expand=True)[[2]]
df_cctv.drop(columns=['소재지지번주소'], inplace=True)

In [14]:
df_cctv[df_cctv['구(동)'].isnull()==True]

,무인교통단속카메라관리번호,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,위도,경도,설치장소,단속구분,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치연도,구(동)
328,10,대구광역시,중구,구도,NaN,종로,3,대구광역시 중구 종로 17,35.867602,128.592078,남성로 정관장앞,4,0,NaN,NaN,NaN,2009.0,NaN
738,G8181,대구광역시,중구,특별시도,NaN,국채보상로,2,대구광역시 중구 국채보상로 713,35.868862,128.608593,동인초교건너(동신교차로→종각네거리),2,50,NaN,NaN,99.0,2020.0,NaN


In [15]:
df_cctv.loc[328, '구(동)'] = '종로'
df_cctv.loc[738, '구(동)'] = '동인동'

In [16]:
df_cctv.rename(columns={'시군구명':'군'}, inplace=True)
use_cols = ['군', '제한속도','설치연도','구(동)']
df_cctv = df_cctv[use_cols]
df_cctv

,군,제한속도,설치연도,구(동)
0,남구,0,2010.0,대명동
1,남구,0,2013.0,봉덕동
2,남구,0,2013.0,봉덕동
3,남구,0,2014.0,대명동
4,남구,0,2014.0,봉덕동
...,...,...,...,...
1060,북구,40,2021.0,구암동
1061,북구,30,2021.0,사수동
1062,북구,30,2021.0,대현동
1063,북구,30,2021.0,대현동


In [17]:
df_cctv.isnull().sum()

군       0
제한속도    0
설치연도    0
구(동)    0
dtype: int64

In [18]:
df_cctv.describe()

,제한속도,설치연도
count,1065.000000,1065.000000
mean,38.281690,2017.824413
std,24.455643,3.600617
min,0.000000,2005.000000
25%,30.000000,2017.000000
50%,50.000000,2019.000000
75%,50.000000,2021.000000
max,110.000000,2021.000000


In [19]:
cctv_gun_limit_avg = pd.DataFrame(df_cctv.groupby(['군'])[['제한속도']].mean()).reset_index()
cctv_gun_limit_avg.rename(columns={'제한속도':'평균제한속도'}, inplace=True)
cctv_gun_limit_avg

,군,평균제한속도
0,군위군,60.416667
1,남구,18.936170
2,달서구,23.862816
3,달성,48.909091
4,달성군,49.823009
5,동구,55.434783
6,북구,45.920000
7,서구,47.272727
8,수성구,50.784314
9,중구,17.808219


In [20]:
cctv_gudong_count = pd.DataFrame(df_cctv.groupby(['군', '설치연도'])[['제한속도']].count()).reset_index()
cctv_gudong_count.rename(columns={'제한속도':'cctv개수'}, inplace=True)
cctv_gudong_count

,군,설치연도,cctv개수
0,군위군,2017.0,1
1,군위군,2018.0,4
2,군위군,2019.0,6
3,군위군,2020.0,6
4,군위군,2021.0,7
...,...,...,...
105,중구,2017.0,7
106,중구,2018.0,7
107,중구,2019.0,2
108,중구,2020.0,4


In [24]:
cctv_info = pd.merge(cctv_gun_limit_avg, cctv_gudong_count, how='left', on=["군", "설치연도"])
cctv_info.rename(columns={'설치연도':'년도'}, inplace=True)
cctv_info

KeyError: '설치연도'

In [23]:
if df_cctv['설치연도'].any() > 2021:
    X_df = pd.merge(df_train, cctv_info, how='left', on=["군", "년도"]).set_index(df_train.index)
else:
    cctv_info[cctv_info['년도']==2021].drop(columns='년도')
    X_df = pd.merge(X_df, cctv_info, how='inner', on=["군"])

NameError: name 'cctv_info' is not defined

In [ ]:
X_df = pd.merge(df_train, cctv_info, how='left', on=["군", "년도"]).set_index(df_train.index)
X_df

# 강수량

In [ ]:
df_rain_info = pd.read_csv("./data/rain_info.csv")
df_rain_info = df_rain_info.T

In [ ]:
df_rain_info.drop(index='강수량', inplace=True)
df_rain_info.reset_index(inplace=True)
df_rain_info.rename(columns={'index':'월', 0:'강수량_합계', 1:'강수량_평균', 2:'강수량_중앙값'}, inplace=True)
df_rain_info

In [ ]:
if '월' in df_rain_info.columns:
    df_rain_info['월'] = df_rain_info['월'].map(lambda x: x[:-1])
    df_rain_info['월'] = df_rain_info['월'].map(lambda x: int(x))
df_rain_info

In [ ]:
X_df = pd.merge(X_df, df_rain_info, how='left', on='월').set_index(X_df.index)
X_df

# 어린이보호구역

In [ ]:
df_child_origin = pd.read_csv("./data/external_open/대구 어린이 보호 구역 정보.csv", encoding='cp949')
df_child = copy.deepcopy(df_child_origin)
df_child

In [ ]:
if '소재지지번주소' in df_child.columns:
  # 군, 구(동)
  df_child[['군', '구(동)']] = df_child['소재지지번주소'].str.split(' ', expand=True)[[1, 2]]
df_child.drop(columns=['소재지지번주소'], inplace=True)

In [ ]:
df_child = df_child[['CCTV설치여부', 'CCTV설치대수', '군', '구(동)']]
df_child = df_child[df_child['CCTV설치여부']=='Y']
df_child.groupby(['군'])[['CCTV설치여부','CCTV설치대수']]
df_child

### def merge_features_from_externals

In [ ]:
import pandas as pd
import numpy as np

train_origin = pd.read_csv("../../main/data/origin/train.csv")
# X_df = train_origin[['사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형','ECLO']]
X_df = train_origin[['사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형']]

### default: 시군구, 사고일시
# 시군구 대전데이터 한정으로, 군, 동 정보만 남긴다.
if '시군구' in X_df.columns:
  # 군, 구(동)
  X_df[['군', '구(동)']] = X_df['시군구'].str.split(' ', expand=True)[[1,2]]
  X_df.drop(columns=['시군구'], inplace=True)
# 사고일시 yyyy-mm-dd hh datetime - 시간 컬럼 생성
if '사고일시' in X_df.columns:
  X_df['년도'] = X_df['사고일시'].str.split('-').str[0].astype(int)
  X_df['월'] = X_df['사고일시'].str.split('-').str[1].astype(int)
  X_df['시간'] = X_df['사고일시'].str.split(' ').str[-1].astype(int)
  X_df.drop(columns=['사고일시'], inplace=True)

### main/data/origin/externals 에서 데이터 가져와서 추가 EDA 진행
## cctv
df_cctv = pd.read_csv("./data/external_open/대구 CCTV 정보.csv", encoding='cp949')

#  address value
if '소재지지번주소' in df_cctv.columns:
  df_cctv['구(동)'] = df_cctv['소재지지번주소'].str.split(' ', expand=True)[[2]]
df_cctv.drop(columns=['소재지지번주소'], inplace=True)
df_cctv.rename(columns={'시군구명':'군'}, inplace=True)

# fill null values
df_cctv.loc[328, '구(동)'] = '종로'
df_cctv.loc[738, '구(동)'] = '동인동'
use_cols = ['군', '제한속도','설치연도','구(동)']
df_cctv = df_cctv[use_cols]

# speed limit for each year&town
cctv_gun_limit_avg = pd.DataFrame(df_cctv.groupby(['군', '설치연도'])[['제한속도']].mean()).reset_index()
cctv_gun_limit_avg.rename(columns={'제한속도':'평균제한속도'}, inplace=True)

# number of cctvs for each year&town
cctv_gudong_count = pd.DataFrame(df_cctv.groupby(['군', '설치연도'])[['제한속도']].count()).reset_index()
cctv_gudong_count.rename(columns={'제한속도':'cctv개수'}, inplace=True)

# merge number of cctvs & speed limit & X_df
cctv_info = pd.merge(cctv_gun_limit_avg, cctv_gudong_count, how='left', on=["군", "설치연도"])
cctv_info.rename(columns={'설치연도':'년도'}, inplace=True)
if (X_df['년도'] > 2021).any():
  cctv_info2 = pd.DataFrame(cctv_info.groupby(['군', '년도'])[['평균제한속도', 'cctv개수']].mean()).reset_index()
  cctv_info2 = cctv_info[cctv_info['년도']==cctv_info['년도'].max()].drop(columns='년도')
  X_df = pd.merge(X_df, cctv_info2, how='left', on=["군"])
else:
  print("2021내")
  X_df = pd.merge(X_df, cctv_info, how='left', on=["군", "년도"]).set_index(X_df.index)

X_df.drop(columns='년도', axis=1, inplace=True)

## rain info
df_rain_info = pd.read_csv("./data/rain_info.csv")
df_rain_info = df_rain_info.T
df_rain_info.drop(index='강수량', inplace=True)
df_rain_info.reset_index(inplace=True)
df_rain_info.rename(columns={'index':'월', 0:'강수량_합계', 1:'강수량_평균', 2:'강수량_중앙값'}, inplace=True)
if '월' in df_rain_info.columns:
  df_rain_info['월'] = df_rain_info['월'].map(lambda x: x[:-1])
  df_rain_info['월'] = df_rain_info['월'].map(lambda x: int(x))
X_df = pd.merge(X_df, df_rain_info, how='left', on='월').set_index(X_df.index)

In [ ]:
X_df